In [5]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 15 14:49:58 2022

@author: Sidi Wu and Cédric Beaulac

Functional autoencoder implementation
"""

# Import modules
import sys
import torch
import torch.nn.init as init
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from numpy import *
import seaborn as sns
import matplotlib
#matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
# import skfda as fda
# from skfda import representation as representation
# from skfda.exploratory.visualization import FPCAPlot
# # from skfda.exploratory.visualization import FPCAPlot
# # from skfda.preprocessing.dim_reduction import FPCA
# # from skfda.representation.basis import BSpline, Fourier, Monomial
import scipy
from scipy.interpolate import BSpline
import os
import ignite
import sklearn
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import random
from random import seed
from scipy import stats
import statistics
from statistics import stdev
import skfda

import skfda
from skfda import FDataGrid as fd
from skfda.representation.basis import BSpline as B


In [6]:
import pandas as pd
import os
os.chdir("C:/Users/Utilisateur/Documents/Master SSD/STAGE RECHERCHE/FDA_python/Functionnal_classifier")
# Lire le fichier CSV
sofa = pd.read_csv('sofa.csv')

# Attribuer des noms de colonnes personnalisés


# Afficher les noms de colonnes
print(sofa.columns)
grid_points=np.arange(173)
sofa.values.shape


Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       ...
       'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 'V171', 'V172', 'V173',
       'V174'],
      dtype='object', length=174)


(520, 174)

In [4]:
Y_X=torch.from_numpy(sofa.values).float().cuda()

y,x=Y_X[:,0].unsqueeze(1).unsqueeze(2).long(),Y_X[:,1:41].reshape(520,1,40)
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(x,y,shuffle=True)
x_train.shape,y_train.shape


(torch.Size([390, 1, 40]), torch.Size([390, 1, 1]))

In [9]:

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 3)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 3)
        torch.nn.init.constant_(m.bias.data, 0.0)


In [709]:
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN,self).__init__()
        self.convlayer1=nn.Sequential(

            nn.Conv1d(1,128,kernel_size=7,stride=1,padding=0,dilation=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.18),
            
        )
        
        self.convlayer2=nn.Sequential(

            nn.Conv1d(128,64,kernel_size=4,stride=1,padding=0,dilation=1),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.18),
            
        )
        
        self.convlayer3=nn.Sequential(

            nn.Conv1d(64,32,kernel_size=7,stride=1,padding=0,dilation=1),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(0.18),
        )


        self.fc_block=nn.Sequential(
            nn.Flatten(),
            nn.Linear(25*32,512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.18),
        
            nn.Linear(512,2),
        )

    def forward(self,x):
        Conv_out=self.convlayer1(x)
        Conv_out2=self.convlayer2(Conv_out)
        Conv_out3=self.convlayer3(Conv_out2)
        n_conv_out=Conv_out3.shape[1]
        Lin_out=self.fc_block(Conv_out3)
        return Lin_out.float().unsqueeze_(2).unsqueeze_(3)


In [674]:
n_conv_out=25
CNN=FCNN().cuda().apply(weights_init_normal)
Conv_out=CNN.convlayer3(CNN.convlayer2(CNN.convlayer1(x_train)))
[x_train.shape,Conv_out.shape]


[torch.Size([390, 1, 40]), torch.Size([390, 32, 25])]

In [714]:
lr_adam=0.001
lr_sgd=0.001
batch_size=78
betas=[0.45,0.98]
# optimizer=optim.Adam(CNN.parameters(),lr=lr_adam,betas=betas)
optimizer=optim.SGD(CNN.parameters(),lr=lr_sgd)
loss_function=nn.CrossEntropyLoss()


In [715]:

from tqdm import tqdm
def train(n_epochs):
    for epoch in range(n_epochs):
        train_loss=torch.tensor(0).cuda().long()
        
        for i in range(int(len(x_train)/batch_size)):
            functions_train=x_train[batch_size*i:batch_size*(1+i),:,:]
            labels_train=y_train[batch_size*i:batch_size*(1+i)]
            optimizer.zero_grad()
            output=CNN(functions_train)
            loss=loss_function(input=output,target=labels_train)
            
            loss.backward()
            optimizer.step()
            train_loss+=loss.long()
        return train_loss,loss 

            


        

In [716]:
print(train(10))
print("Précision moyenne =",((torch.sum(torch.argmax(CNN(x_test),dim=1)==y_test)/x_test.shape[0])*100).detach().cpu().numpy(),"%")

(tensor(0, device='cuda:0'), tensor(0.1249, device='cuda:0', grad_fn=<NllLoss2DBackward0>))
Précision moyenne = 82.30769 %


In [717]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

0